# Google Play store's App Marketplace team

In [1]:
import pandas as pd 
import numpy as np  

In [2]:
df_dim_app = pd.read_csv('Data/002/dim_app.csv')
df_browsing = pd.read_csv('Data/002/fct_app_browsing.csv')
df_downloads = pd.read_csv('Data/002/fct_app_downloads.csv')

In [4]:
df_dim_app.head()

,app_id,app_name,category,app_type
0,1,FunPlay,Games,Free
1,2,TaskMaster,Productivity,Premium
2,3,HealthPlus,Health,Free
3,4,EduLearn,Education,Premium
4,5,StreamFun,Entertainment,Free


In [5]:
df_browsing.head()

,app_id,browse_date,browse_count
0,1,2024-10-05,150
1,2,2024-10-15,200
2,3,2024-11-01,0
3,4,2024-11-20,120
4,5,2024-12-10,300


In [6]:
df_downloads.head()

,app_id,download_date,download_count
0,1,2024-10-06,30
1,2,2024-10-16,50
2,3,2024-11-02,0
3,4,2024-11-21,40
4,5,2024-12-11,90


# Pregunta 1

### El equipo de marketplace desea identificar las categorías de aplicaciones con alto y bajo rendimiento. Proporciona el total de descargas por categoría para el mes de noviembre de 2024. Si no hubo descargas para una categoría, devuelve el valor como 0.

In [7]:
# 1. Convertir la columna de fecha a formato datetime (si no lo está ya)
df_downloads['download_date'] = pd.to_datetime(df_downloads['download_date'])

# 2. Filtrar las descargas solo para noviembre de 2024
# Esto equivale al filtro que pusiste en el 'ON' del JOIN
downloads_nov = df_downloads[
    (df_downloads['download_date'] >= '2024-11-01') & 
    (df_downloads['download_date'] <= '2024-11-30')
]

# 3. Hacer el LEFT JOIN entre la dimensión y las descargas filtradas
# how='left' asegura que mantengamos todas las categorías de df_dim_app
merged_df = pd.merge(df_dim_app, downloads_nov, on='app_id', how='left')

# 4. Agrupar por categoría, sumar y manejar los valores nulos (COALESCE)
result = (
    merged_df.groupby('category')['download_count']
    .sum()                # Sumamos las descargas
    .fillna(0)            # COALESCE: convierte NaN en 0
    .reset_index()        # Convierte el resultado en un DataFrame limpio
    .sort_values(by='download_count', ascending=False) # ORDER BY desc
)

print(result)

         category  download_count
1       Education            40.0
11         Social            40.0
14        Weather            18.0
13      Utilities            15.0
3         Finance             0.0
0           Books             0.0
2   Entertainment             0.0
6       Lifestyle             0.0
5          Health             0.0
4           Games             0.0
7           Music             0.0
10   Productivity             0.0
9     Photography             0.0
8            News             0.0
12         Travel             0.0


```SQL
SELECT
    a.category,
    COALESCE(SUM(d.download_count),0) AS total_downloads
FROM dim_app a
LEFT JOIN fct_app_downloads d
    ON a.app_id = d.app_id
    AND d.download_date BETWEEN '2024-11-01' AND '2024-11-30'
GROUP BY a.category
ORDER BY total_downloads DESC;
```

# Pregunta 2

### El objetivo de nuestro equipo es la tasa de conversión de descargas, definida como descargas por cada evento de navegación. Para cada categoría de aplicación, calcula la tasa de conversión de descargas en diciembre, eliminando las categorías donde los conteos de navegación sean cero.

In [8]:
# 1. Asegurar formato de fecha
df_browsing['browse_date'] = pd.to_datetime(df_browsing['browse_date'])
df_downloads['download_date'] = pd.to_datetime(df_downloads['download_date'])

# 2. Filtrar solo datos de diciembre 2024
dec_browsing = df_browsing[df_browsing['browse_date'].dt.month == 12]
dec_downloads = df_downloads[df_downloads['download_date'].dt.month == 12]

# 3. Agrupar por app_id para tener totales únicos por aplicación en diciembre
agg_browsing = dec_browsing.groupby('app_id')['browse_count'].sum().reset_index()
agg_downloads = dec_downloads.groupby('app_id')['download_count'].sum().reset_index()

# 4. Unir la dimensión de apps con la navegación (INNER JOIN para cumplir con "remover si navegación es cero")
df_merged = pd.merge(df_dim_app, agg_browsing, on='app_id', how='inner')

# 5. Unir con las descargas (LEFT JOIN para no perder apps que tuvieron navegación pero 0 descargas)
df_merged = pd.merge(df_merged, agg_downloads, on='app_id', how='left').fillna(0)

# 6. Agrupar por categoría y calcular la tasa
category_stats = df_merged.groupby('category').agg({
    'download_count': 'sum',
    'browse_count': 'sum'
}).reset_index()

# 7. Calcular Conversion Rate y ordenar
category_stats['conversion_rate'] = category_stats['download_count'] / category_stats['browse_count']
result = category_stats[['category', 'conversion_rate']].sort_values(by='conversion_rate', ascending=False)

print(result)

        category  conversion_rate
0          Books         0.357143
2  Entertainment         0.300000
1      Education         0.272727
3        Finance         0.250000
4          Games         0.250000
5   Productivity         0.233333
6         Social         0.180000


```SQL
SELECT
    a.category,
    SUM(d.download_count)::FLOAT / SUM(b.browse_count) AS conversion_rate
FROM dim_app a
JOIN fct_app_browsing b ON a.app_id = b.app_id
JOIN fct_app_downloads d ON a.app_id = d.app_id
WHERE b.browse_date BETWEEN '2024-12-01' AND '2024-12-31'
    AND d.download_date BETWEEN '2024-12-01' AND '2024-12-31'
GROUP BY a.category
HAVING SUM(b.browse_count) > 0
ORDER BY conversion_rate DESC;
```

# Pregunta 3

### El equipo desea comparar las tasas de conversión entre aplicaciones gratuitas (Free) y premium en todas las categorías. Combina los datos de conversión de ambos tipos de aplicaciones para presentar una vista unificada para el cuarto trimestre (Q4) de 2024.

In [9]:
# 1. Preparar las fechas
df_browsing['browse_date'] = pd.to_datetime(df_browsing['browse_date'])
df_downloads['download_date'] = pd.to_datetime(df_downloads['download_date'])

# 2. Definir el rango del Q4 2024
inicio_q4 = '2024-10-01'
fin_q4 = '2024-12-31'

# 3. Filtrar datos del Q4
q4_browsing = df_browsing[df_browsing['browse_date'].between(inicio_q4, fin_q4)]
q4_downloads = df_downloads[df_downloads['download_date'].between(inicio_q4, fin_q4)]

# 4. Agrupar totales por app_id para evitar duplicados en el merge
agg_browsing = q4_browsing.groupby('app_id')['browse_count'].sum().reset_index()
agg_downloads = q4_downloads.groupby('app_id')['download_count'].sum().reset_index()

# 5. Unir con la dimensión de apps para obtener el 'app_type'
# Usamos un merge secuencial
df_q4 = pd.merge(df_dim_app, agg_browsing, on='app_id', how='inner')
df_q4 = pd.merge(df_q4, agg_downloads, on='app_id', how='left').fillna(0)

# 6. Agrupar por 'app_type' y calcular la tasa de conversión total
type_stats = df_q4.groupby('app_type').agg({
    'download_count': 'sum',
    'browse_count': 'sum'
}).reset_index()

type_stats['conversion_rate'] = type_stats['download_count'] / type_stats['browse_count']

# 7. Vista unificada final
result = type_stats[['app_type', 'conversion_rate']].sort_values(by='conversion_rate', ascending=False)

print(result)

  app_type  conversion_rate
1  Premium         0.275000
0     Free         0.236667


```SQL
SELECT
    a.app_type,
    SUM(COALESCE(d.total_downloads, 0))::FLOAT / SUM(b.total_browses) AS conversion_rate
FROM dim_app a
JOIN (
    -- Navegación total por app en el Q4 (Oct-Dic)
    SELECT app_id, SUM(browse_count) AS total_browses
    FROM fct_app_browsing
    WHERE browse_date BETWEEN '2024-10-01' AND '2024-12-31'
    GROUP BY app_id
) b ON a.app_id = b.app_id
LEFT JOIN (
    -- Descargas totales por app en el Q4 (Oct-Dic)
    SELECT app_id, SUM(download_count) AS total_downloads
    FROM fct_app_downloads
    WHERE download_date BETWEEN '2024-10-01' AND '2024-12-31'
    GROUP BY app_id
) d ON a.app_id = d.app_id
GROUP BY a.app_type
ORDER BY conversion_rate DESC;
```